In [ ]:
import random
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

TITLE = 'Übung: Reihen- und Parallelschaltung'
E12 = [1.0, 1.2, 1.5, 1.8, 2.2, 2.7, 3.3, 3.9, 4.7, 5.6, 6.8, 8.2]

UNITS = {
    'R': [('mOhm', 1e-3), ('Ohm', 1.0), ('kOhm', 1e3), ('MOhm', 1e6)],
    'L': [('uH', 1e-6), ('mH', 1e-3), ('H', 1.0)],
    'C': [('pF', 1e-12), ('nF', 1e-9), ('uF', 1e-6), ('mF', 1e-3)],
}

TOPOS = [
    {
        'id': '2S',
        'n': 2,
        'name': '2 in Reihe',
        'tree': ('S', [0, 1]),
        'net': [
            '{E}1 A n1; right=2',
            '{E}2 n1 B; right=2',
        ],
    },
    {
        'id': '2P',
        'n': 2,
        'name': '2 parallel',
        'tree': ('P', [0, 1]),
        'net': [
            'W1 A nupL; up=1',
            'W2 A ndnL; down=1',
            '{E}1 nupL nupR; right=2',
            '{E}2 ndnL ndnR; right=2',
            'W3 nupR B; down=1',
            'W4 ndnR B; up=1',
        ],
    },
    {
        'id': '3S',
        'n': 3,
        'name': '3 in Reihe',
        'tree': ('S', [0, 1, 2]),
        'net': [
            '{E}1 A n1; right=1.6',
            '{E}2 n1 n2; right=1.6',
            '{E}3 n2 B; right=1.6',
        ],
    },
    {
        'id': '3P',
        'n': 3,
        'name': '3 parallel',
        'tree': ('P', [0, 1, 2]),
        'net': [
            'W1 A ntL; up=1.2',
            'W2 A nmL; right=0.1',
            'W3 A nbL; down=1.2',
            '{E}1 ntL ntR; right=2',
            '{E}2 nmL nmR; right=2',
            '{E}3 nbL nbR; right=2',
            'W4 ntR B; down=1.2',
            'W5 nmR B; right=0.1',
            'W6 nbR B; up=1.2',
        ],
    },
    {
        'id': '3S_P',
        'n': 3,
        'name': '1 in Reihe, 2 parallel',
        'tree': ('S', [0, ('P', [1, 2])]),
        'net': [
            '{E}1 A n1; right=2',
            'W1 n1 nupL; up=1',
            'W2 n1 ndnL; down=1',
            '{E}2 nupL nupR; right=2',
            '{E}3 ndnL ndnR; right=2',
            'W3 nupR B; down=1',
            'W4 ndnR B; up=1',
        ],
    },
    {
        'id': '3P_S',
        'n': 3,
        'name': '2 parallel, dann 1 in Reihe',
        'tree': ('S', [('P', [0, 1]), 2]),
        'net': [
            'W1 A nupL; up=1',
            'W2 A ndnL; down=1',
            '{E}1 nupL nupR; right=2',
            '{E}2 ndnL ndnR; right=2',
            'W3 nupR n1; down=1',
            'W4 ndnR n1; up=1',
            '{E}3 n1 B; right=2',
        ],
    },
    {
        'id': '4S',
        'n': 4,
        'name': '4 in Reihe',
        'tree': ('S', [0, 1, 2, 3]),
        'net': [
            '{E}1 A n1; right=1.3',
            '{E}2 n1 n2; right=1.3',
            '{E}3 n2 n3; right=1.3',
            '{E}4 n3 B; right=1.3',
        ],
    },
    {
        'id': '4P',
        'n': 4,
        'name': '4 parallel',
        'tree': ('P', [0, 1, 2, 3]),
        'net': [
            'W1 A n1L; up=1.5',
            'W2 A n2L; up=0.5',
            'W3 A n3L; down=0.5',
            'W4 A n4L; down=1.5',
            '{E}1 n1L n1R; right=2',
            '{E}2 n2L n2R; right=2',
            '{E}3 n3L n3R; right=2',
            '{E}4 n4L n4R; right=2',
            'W5 n1R B; down=1.5',
            'W6 n2R B; down=0.5',
            'W7 n3R B; up=0.5',
            'W8 n4R B; up=1.5',
        ],
    },
    {
        'id': '4SPP',
        'n': 4,
        'name': '1 in Reihe + 3 parallel',
        'tree': ('S', [0, ('P', [1, 2, 3])]),
        'net': [
            '{E}1 A n1; right=1.6',
            'W1 n1 ntL; up=1.2',
            'W2 n1 nmL; right=0.1',
            'W3 n1 nbL; down=1.2',
            '{E}2 ntL ntR; right=2',
            '{E}3 nmL nmR; right=2',
            '{E}4 nbL nbR; right=2',
            'W4 ntR B; down=1.2',
            'W5 nmR B; right=0.1',
            'W6 nbR B; up=1.2',
        ],
    },

    {
        'id': '4SPP_R',
        'n': 4,
        'name': '3 parallel + 1 in Reihe',
        'tree': ('S', [('P', [0, 1, 2]), 3]),
        'net': [
            'W1 A ntL; up=1.2',
            'W2 A nmL; right=0.1',
            'W3 A nbL; down=1.2',
            '{E}1 ntL ntR; right=2',
            '{E}2 nmL nmR; right=2',
            '{E}3 nbL nbR; right=2',
            'W4 ntR n1; down=1.2',
            'W5 nmR n1; right=0.1',
            'W6 nbR n1; up=1.2',
            '{E}4 n1 B; right=1.6',
        ],
    },
    {
        'id': '4PSS',
        'n': 4,
        'name': '3 in Reihe parallel zu 1',
        'tree': ('P', [('S', [0, 1, 2]), 3]),
        'net': [
            '{E}1 A n1; right=1.4',
            '{E}2 n1 n2; right=1.4',
            '{E}3 n2 B; right=1.4',
            'W1 A nbL; down=1.2',
            '{E}4 nbL nbR; right=4.2',
            'W2 nbR B; up=1.2',
        ],
    },
    {
        'id': '4SP2P2',
        'n': 4,
        'name': '2 parallel in Reihe zu 2 parallel',
        'tree': ('S', [('P', [0, 1]), ('P', [2, 3])]),
        'net': [
            'W1 A u1L; up=1',
            'W2 A d1L; down=1',
            '{E}1 u1L u1R; right=1.8',
            '{E}2 d1L d1R; right=1.8',
            'W3 u1R n1; down=1',
            'W4 d1R n1; up=1',
            'W5 n1 u2L; up=1',
            'W6 n1 d2L; down=1',
            '{E}3 u2L u2R; right=1.8',
            '{E}4 d2L d2R; right=1.8',
            'W7 u2R B; down=1',
            'W8 d2R B; up=1',
        ],
    },
    {
        'id': '4PS2S2',
        'n': 4,
        'name': '2 in Reihe parallel zu 2 in Reihe',
        'tree': ('P', [('S', [0, 1]), ('S', [2, 3])]),
        'net': [
            '{E}1 A n1; right=1.6',
            '{E}2 n1 B; right=1.6',
            'W1 A dL; down=1.2',
            '{E}3 dL dM; right=1.6',
            '{E}4 dM dR; right=1.6',
            'W2 dR B; up=1.2',
        ],
    },
    {
        'id': '4S_MIX_L',
        'n': 4,
        'name': '2 parallel + 2 in Reihe',
        'tree': ('S', [('P', [0, 1]), 2, 3]),
        'net': [
            'W1 A uL; up=1',
            'W2 A dL; down=1',
            '{E}1 uL uR; right=1.8',
            '{E}2 dL dR; right=1.8',
            'W3 uR n1; down=1',
            'W4 dR n1; up=1',
            '{E}3 n1 n2; right=1.3',
            '{E}4 n2 B; right=1.3',
        ],
    },
    {
        'id': '4S_MIX_R',
        'n': 4,
        'name': '2 in Reihe + 2 parallel',
        'tree': ('S', [0, 1, ('P', [2, 3])]),
        'net': [
            '{E}1 A n1; right=1.3',
            '{E}2 n1 n2; right=1.3',
            'W1 n2 uL; up=1',
            'W2 n2 dL; down=1',
            '{E}3 uL uR; right=1.8',
            '{E}4 dL dR; right=1.8',
            'W3 uR B; down=1',
            'W4 dR B; up=1',
        ],
    },
    {
        'id': '4P_MIX_L',
        'n': 4,
        'name': '2 in Reihe parallel zu 2 Einzelzweigen',
        'tree': ('P', [('S', [0, 1]), 2, 3]),
        'net': [
            'W1 A uL; up=1.2',
            'W2 A mL; right=0.1',
            'W3 A dL; down=1.2',
            '{E}1 uL uM; right=1.0',
            '{E}2 uM uR; right=1.0',
            '{E}3 mL mR; right=2',
            '{E}4 dL dR; right=2',
            'W4 uR B; down=1.2',
            'W5 mR B; right=0.1',
            'W6 dR B; up=1.2',
        ],
    },
    {
        'id': '4P_MIX_R',
        'n': 4,
        'name': '2 Einzelzweige parallel zu 2 in Reihe',
        'tree': ('P', [0, 1, ('S', [2, 3])]),
        'net': [
            'W1 A uL; up=1.2',
            'W2 A mL; right=0.1',
            'W3 A dL; down=1.2',
            '{E}1 uL uR; right=2',
            '{E}2 mL mR; right=2',
            '{E}3 dL dM; right=1.0',
            '{E}4 dM dR; right=1.0',
            'W4 uR B; down=1.2',
            'W5 mR B; right=0.1',
            'W6 dR B; up=1.2',
        ],
    },
]

state = {}


def stable_seed(seed_text):
    t = str(seed_text).strip()
    if t.isdigit():
        return int(t)
    return random.randint(100000, 999999)


def fmt_num(x, sig=4):
    s = f'{x:.{sig}g}'
    if 'e' not in s and '.' in s:
        s = s.rstrip('0').rstrip('.')
    return s


def eq_value(kind, tree, vals):
    if isinstance(tree, int):
        return vals[tree]
    op, items = tree
    sub = [eq_value(kind, t, vals) for t in items]
    if kind in ('R', 'L'):
        if op == 'S':
            return sum(sub)
        return 1.0 / sum(1.0 / x for x in sub)
    # C
    if op == 'P':
        return sum(sub)
    return 1.0 / sum(1.0 / x for x in sub)


def kind_name(kind):
    if kind == 'R':
        return 'Ersatzwiderstand'
    if kind == 'L':
        return 'Ersatzinduktivität'
    return 'Ersatzkapazität'


def symbol(kind):
    return {'R': 'R', 'L': 'L', 'C': 'C'}[kind]


def base_unit(kind):
    return {'R': 'Ohm', 'L': 'H', 'C': 'F'}[kind]


def make_netlist(kind, topo):
    lines = [ln.replace('{E}', kind) for ln in topo['net']]
    lines.append('WA A a_ext; left=0.9, l=A')
    lines.append('WB B b_ext; right=0.9, l=B')
    return '\n'.join(lines) + '\n'


def make_case(seed_text=''):
    seed = stable_seed(seed_text)
    rng = random.Random(seed)
    kind = rng.choice(['R', 'L', 'C'])
    topos = [t for t in TOPOS if 2 <= t['n'] <= 4]
    topo = rng.choice(topos)

    unit_label, unit_mult = rng.choice(UNITS[kind])
    coeffs = [rng.choice(E12) for _ in range(topo['n'])]
    vals = [c * unit_mult for c in coeffs]

    eq = eq_value(kind, topo['tree'], vals)

    return {
        'seed': seed,
        'kind': kind,
        'topo': topo,
        'unit_label': unit_label,
        'unit_mult': unit_mult,
        'coeffs': coeffs,
        'vals': vals,
        'eq': eq,
    }


def render_schematic(c):
    try:
        from lcapy import Circuit
    except Exception:
        return None, 'lcapy/LaTeX nicht verfügbar.'
    try:
        out_dir = Path('schematics/parallel_series')
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / f"parallel_series_{c['seed']}.png"
        cct = Circuit(make_netlist(c['kind'], c['topo']))
        for i, v in enumerate(c['vals'], start=1):
            cct[f"{c['kind']}{i}"].value = v
        cct.draw(str(out_path), style='european', label_values=True, label_nodes=False, draw_nodes=False, node_spacing=1.2, cpt_size=1.1)
        return out_path, ''
    except Exception as exc:
        return None, f'Schaltplan konnte nicht erzeugt werden: {exc}'


title = widgets.HTML(f'<h2>{TITLE}</h2>')
seed = widgets.Text(description='Seed:', value='')
btn_seed = widgets.Button(description='Generieren (Seed)')
btn_rand = widgets.Button(description='Generieren (Random)')

out_given = widgets.HTML()
out_task = widgets.HTML()
img = widgets.Image(format='png', layout=widgets.Layout(max_width='600px', max_height='600px'))
out_img = widgets.HTML()

ans_label = widgets.HTML('')
inp_eq = widgets.FloatText(description='', layout=widgets.Layout(border='1px solid #d0d7de'))
btn_check = widgets.Button(description='Prüfen')
out_check = widgets.HTML()


def refresh(c):
    s = symbol(c['kind'])
    lines = [
        '<b>Gegeben</b><br>',
        f"- Seed: {c['seed']}<br>",
        f"- Bauteiltyp: {c['kind']}<br>",
        f"- Topologie: {c['topo']['name']}<br>",
        f"- Einheit (alle gleich): {c['unit_label']}<br>",
    ]
    for i, cf in enumerate(c['coeffs'], start=1):
        lines.append(f"- {s}<sub>{i}</sub> = {fmt_num(cf)} {c['unit_label']}<br>")
    out_given.value = ''.join(lines)

    out_task.value = (
        '<b>Aufgabe</b><br>'
        f"Berechne die {kind_name(c['kind'])} zwischen den Klemmen A-B.<br>"
        f"Gib das Ergebnis in SI-Basiseinheit an ({base_unit(c['kind'])}).<br>"
        f"{s}<sub>AB</sub>"
    )

    ans_label.value = f"<b>{s}<sub>AB</sub> [{base_unit(c['kind'])}]:</b>"


def on_generate(seed_text=''):
    c = make_case(seed_text)
    state.clear()
    state.update(c)
    refresh(c)
    p, msg = render_schematic(c)
    if p and p.exists():
        img.value = p.read_bytes()
        out_img.value = ''
    else:
        img.value = b''
        out_img.value = f'<span style="color:#cf222e">{msg}</span>' if msg else ''
    out_check.value = ''
    inp_eq.layout.border = '1px solid #d0d7de'


def on_seed(_):
    on_generate(seed.value)


def on_rand(_):
    seed.value = str(random.randint(100000, 999999))
    on_generate(seed.value)


def on_check(_):
    if not state:
        return

    # Prüfung eindeutig in SI-Basiseinheit
    ref_si = state['eq']

    try:
        user = float(inp_eq.value)
    except Exception:
        out_check.value = '<span style="color:#cf222e"><b>Ungültige Eingabe.</b></span>'
        inp_eq.layout.border = '2px solid #cf222e'
        return

    tol = max(1e-12, 0.01 * abs(ref_si))
    ok = abs(user - ref_si) <= tol

    si_unit = base_unit(state['kind'])
    expected_si = f"{fmt_num(ref_si)} {si_unit}"
    expected_pref = f"{fmt_num(ref_si / state['unit_mult'])} {state['unit_label']}"

    if ok:
        out_check.value = (
            f'<span style="color:#1a7f37"><b>Richtig.</b> '
            f'Erwartet: {expected_si} (entspricht {expected_pref})</span>'
        )
        inp_eq.layout.border = '2px solid #1a7f37'
    else:
        out_check.value = (
            f'<span style="color:#cf222e"><b>Falsch.</b> '
            f'Erwartet: {expected_si} (entspricht {expected_pref})</span>'
        )
        inp_eq.layout.border = '2px solid #cf222e'


btn_seed.on_click(on_seed)
btn_rand.on_click(on_rand)
btn_check.on_click(on_check)

ui = widgets.VBox([
    title,
    widgets.HBox([seed, btn_seed, btn_rand]),
    out_given,
    out_task,
    img,
    out_img,
    widgets.HBox([ans_label, inp_eq, btn_check]),
    out_check,
])

display(ui)
on_rand(None)
